In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
import os

In [ ]:
# Define paths
zip_path = "/content/drive/MyDrive/raw_data/PRAICP-1001-GenderDetc.zip"  # Change to your uploaded file name
extract_path = "/content/drive/MyDrive/gender"

# Extract ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Check if extraction was successful
print("Extracted files:", os.listdir(extract_path))

Extracted files: ['Data']


In [ ]:
male_path = "/content/drive/MyDrive/gender/Data/gender_dataset_face/man"
female_path = "/content/drive/MyDrive/gender/Data/gender_dataset_face/woman"

print(f"Number of Male images: {len(os.listdir(male_path))}")
print(f"Number of Female images: {len(os.listdir(female_path))}")


Number of Male images: 1174
Number of Female images: 1135


In [ ]:
import os

# Count the number of images in each folder
male_count = len(os.listdir(male_path))
female_count = len(os.listdir(female_path))

print(f"Number of Male images: {male_count}")
print(f"Number of Female images: {female_count}")

# Find the difference
difference = abs(male_count - female_count)
print(f"Difference: {difference}")


Number of Male images: 1174
Number of Female images: 1135
Difference: 39


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import numpy as np
import os
from PIL import UnidentifiedImageError  # Import the specific error

# Define augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=30,  # Rotate images randomly
    width_shift_range=0.2,  # Shift images horizontally
    height_shift_range=0.2,  # Shift images vertically
    shear_range=0.2,  # Apply shear transformation
    zoom_range=0.2,  # Zoom into images
    horizontal_flip=True,  # Flip images horizontally
    fill_mode='nearest'  # Fill missing pixels
)

def augment_images(folder, target_count):
    current_count = len(os.listdir(folder))
    images_needed = target_count - current_count

    print(f"Generating {images_needed} new images for {folder}...")

    for filename in os.listdir(folder):
        if images_needed <= 0:
            break  # Stop if we have enough images

        img_path = os.path.join(folder, filename)

        # Add error handling for image loading
        try:
            img = load_img(img_path, target_size=(224, 224))  # Load and resize
        except UnidentifiedImageError:
            print(f"Skipping file {filename} - Could not be opened as an image.")
            continue  # Move to the next file

        img_array = img_to_array(img)  # Convert to NumPy array
        img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions

        # Generate 5 augmented images per original image
        i = 0
        for batch in datagen.flow(img_array, batch_size=1, save_to_dir=folder, save_prefix="aug", save_format="jpg"):
            i += 1
            images_needed -= 1
            if images_needed <= 0 or i >= 5:
                break

# Determine which class needs augmentation
if male_count < female_count:
    augment_images(male_path, female_count)  # Increase Male images
else:
    augment_images(female_path, male_count)  # Increase Female images

print("Augmentation completed!")

Generating 39 new images for /content/drive/MyDrive/gender/Data/gender_dataset_face/woman...
Skipping file .DS_Store - Could not be opened as an image.
Augmentation completed!


In [ ]:
print(f"Updated Male images: {len(os.listdir(male_path))}")
print(f"Updated Female images: {len(os.listdir(female_path))}")

Updated Male images: 1174
Updated Female images: 1174


In [ ]:
# Build the CNN model
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

# Load the MobileNetV2 model (pre-trained on ImageNet), excluding the top layers
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the base model to retain pre-trained weights
base_model.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Convert feature maps to a single vector
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)  # Prevent overfitting
# The output layer needs to be called on the previous layer's output to create a KerasTensor
output = Dense(1, activation='sigmoid')(x)  # Binary classification (Male/Female)

# Define the model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),  # Lower LR for fine-tuning
              loss='binary_crossentropy',
              metrics=['accuracy'])
print("Model Compiled successfully")

# Summary of the model
model.summary()

Model Compiled successfully


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 112, 112, 32)   │            864 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 112, 112, 32)   │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 112, 112, 32)   │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 112, 112, 32)   │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 112, 112, 16)   │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 112, 112, 16)   │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 112, 112, 96)   │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 112, 112, 96)   │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 112, 112, 96)   │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 113, 113, 96)   │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 56, 56, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├──────────────────────

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
# Train the model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator for training with normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
)

# Set up the training data generator
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/gender/Data/gender_dataset_face',  # Replace with the path to your training dataset
    target_size=(224, 224),  # Resize images to match input size
    batch_size=32,
    class_mode='binary',  # Binary classification: male/female
    shuffle=True
)

# Train the model
model.fit(train_generator, epochs=10)

# Save the trained model
model.save("gender_classification_model.keras")

print("Model training and saving completed successfully!")

Found 2346 images belonging to 2 classes.
Epoch 1/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 111s 1s/step - accuracy: 0.9332 - loss: 0.1771
Epoch 2/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - accuracy: 0.9389 - loss: 0.1754
Epoch 3/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - accuracy: 0.9326 - loss: 0.1654
Epoch 4/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - accuracy: 0.9377 - loss: 0.1549
Epoch 5/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 145s 1s/step - accuracy: 0.9418 - loss: 0.1558
Epoch 6/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - accuracy: 0.9474 - loss: 0.1419
Epoch 7/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - accuracy: 0.9587 - loss: 0.1251
Epoch 8/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 111s 2s/step - accuracy: 0.9495 - loss: 0.1308
Epoch 9/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 108s 1s/step - accuracy: 0.9640 - loss: 0.1168
Epoch 10/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 108s 1s/step - accuracy: 0.9602 - loss: 0.1164
Model training and saving completed successfully!


In [ ]:
# Model Evalution
# Split the data for validation using validation_split
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    validation_split=0.2  # Reserve 20% of the data for validation
)

# Set up the training data generator
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/gender/Data/gender_dataset_face',  # Replace with the path to your training dataset
    target_size=(224, 224),  # Resize images to match input size
    batch_size=32,
    class_mode='binary',  # Binary classification: male/female
    shuffle=True,
    subset='training'  # Indicate this is for training
)

# Set up the validation data generator
validation_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/gender/Data/gender_dataset_face',  # Replace with the path to your training dataset
    target_size=(224, 224),  # Resize images to match input size
    batch_size=32,
    class_mode='binary',  # Binary classification: male/female
    shuffle=True,
    subset='validation'  # Indicate this is for validation
)

# Evaluate the model
evaluation_results = model.evaluate(validation_generator)
print(f"Evaluation results - Loss: {evaluation_results[0]}, Accuracy: {evaluation_results[1]}")

Found 1878 images belonging to 2 classes.
Found 468 images belonging to 2 classes.
15/15 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.9576 - loss: 0.1274
Evaluation results - Loss: 0.113360196352005, Accuracy: 0.9636752009391785


In [1]:
model.save("gender_classification_model_final.keras")
print("model is ready for integration !")

NameError: name 'model' is not defined